# Data Quality Accessment

In [1]:
# Loading crime csv to RDD
# Replace with your directory
crime_csv = sc.textFile('file:/Users/zhuorulin/Documents/DataScience/datasets/NYPD_Complaint_Data_Historic.csv',use_unicode=False)

In [35]:
# Convert csv to DataFrame
from csv import reader # Warning: csv.reader does not support unicode decode
from pyspark.sql import SQLContext
from collections import defaultdict
import datetime
import re
import pandas as pd

In [36]:
# Use csv.reader to read raw binary
lines_rdd = crime_csv.mapPartitions(reader)\
.map(lambda line: [x.decode('utf-8') for x in line] )# Decode with utf-8 codec
# store columns values. Also a search table for searching column name using index
schemas = lines_rdd.take(1)[0]
# filter out first row
lines = lines_rdd.filter(lambda x: x!=schemas)

In [253]:
# This dict makes it easier to search for column index using column name
colname2idx = defaultdict()
for idx,colname in enumerate(schemas):
    colname2idx[colname] = idx
# Example
print(colname2idx['CMPLNT_TO_DT'])

3


# Global Null Checking method
We used following null checking procedure to check all columns:
- Check whether the length of string is zero
- Check whether the string is 'nan'

In [246]:
def checkNull(string):
    #Assume Unicode String
    # Step 1: Check for length 0 i.e '' field
    if len(string.strip())==0:
        return 'NULL'
    # Step 2: Check for 'nan'
    elif string=='nan':
        return 'NULL'
    else:
        return 'VALID'

In [254]:
# Example: Check row
NULL_TABLE = lines.map(lambda line:[checkNull(x) for x in line])
df_nullrow = pd.DataFrame()
df_nullrow['sample_row'] = lines.take(1)[0]
df_nullrow['nullity'] = NULL_TABLE.take(1)[0]
print df_nullrow.head(10)

                       sample_row nullity
0                       101109527   VALID
1                      12/31/2015   VALID
2                        23:45:00   VALID
3                                    NULL
4                                    NULL
5                      12/31/2015   VALID
6                             113   VALID
7                         FORGERY   VALID
8                             729   VALID
9  FORGERY,ETC.,UNCLASSIFIED-FELO   VALID


# Date Format Checking Method
Format Reference: http://strftime.org/

We use python datetime module to check whether date column is of the correct date format.

- If a string is of the wrong format datetime.datetime.strptime() raises error.
- If a year is earlier than 2005 we also consider it to be invalid because this table is designed to be the crime statistics from 2005 to 2015

In [25]:
# Example of error
date_format = '%m/%d/%Y'
testdate = '13/09/1992'
testdate = datetime.datetime.strptime(testdate,date_format)

ValueError: time data '13/09/1992' does not match format '%m/%d/%Y'

In [161]:
date_format = '%m/%d/%Y'
time_format = '%H:%M:%S'
# If date is in wrong format it would not be sucessfully converted to datetime object.
def checkDate(line,date_format=date_format):
    if checkNull(line)=='VALID':
        try:
            date = datetime.datetime.strptime(line,date_format)
            if (date.year <=2015)&(date.year>=2005):
                return 'VALID'
            else:
                return 'INVALID'
        except:
            return 'INVALID'
    else:
        return 'NULL'
def checkTime(line,time_format=time_format):
    if checkNull(line)=='VALID':
        try:
            datetime.datetime.strptime(line,time_format)
            return 'VALID'
        except:
            return 'INVALID'
    else:
        return 'NULL'
########################################

In [252]:
checkDate('10/09/1015')

'INVALID'

### Data Quality Issue
We observe over 19000 invalid date from CMPLNT_FR_DT.

In [27]:
# Testing Check Date Method
CMPLNT_FR_DT.countByValue()

defaultdict(int, {'INVALID': 8670, 'VALID': 5092561})

# Regular Expression Checking Method
Regular Expression Reference: https://docs.python.org/2/library/re.html

In [7]:
# Example Checking KY_CD 3-digit code
regex_3_digits = '^\d{3}$'

In [41]:
#Require re package
def checkRegex(line,regex):
    # Input
    # line: a string for check
    # regex: regular expression pattern
    match = re.match(regex,line)
    if match:
        return 'VALID'
    else:
        return 'INVALID'

In [12]:
# Show that it actually work
print checkRegex('1234',regex_3_digits)
print checkRegex('123',regex_3_digits)

INVALID
VALID


# Data validity checking for each columns

# CMPLNT_NUM
- check nullity
- check duplicate

In [193]:
duplicate_check = lines.map(lambda x:x[colname2idx['CMPLNT_NUM']]).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)

In [195]:
#No duplication
duplicate_check.filter(lambda x: x[1]>1).collect()

[]

In [211]:
# check nullity
lines.map(lambda x:x[colname2idx['CMPLNT_NUM']]).map(checkNull).countByValue()

defaultdict(int, {'VALID': 5101231})

## Numerical Variables
### Latitude/Longitude
- check Nullity
- check whether string can be converted to float
- check whether latitude and longitude is in the range of new york city. (Latitude in [40.48, 40.9] and Longitude in [-74.3, -73.6]) 

Functions use to check longitude and latitude are stored in utilities.py.

In [255]:
%run ./crime-data-process/code/utilities.py

In [256]:
Latitude = lines.map(lambda x: x[colname2idx['Latitude']])
checkResults_Latitude = Latitude.map(checkLatitude)

In [257]:
checkResults_Latitude.countByValue()

defaultdict(int, {'INVALID': 11251, 'NULL': 188146, 'VALID': 4901834})

In [258]:
Longitude = lines.map(lambda x: x[colname2idx['Longitude']])
checkResults_Longitude = Longitude.map(checkLongitude)
checkResults_Longitude.countByValue()

defaultdict(int, {'NULL': 188146, 'VALID': 4913085})

# X_COORD_CD/Y_COORD_CD

- Check Nullity only

In [251]:
X_COORD_CD = lines.map(lambda x: x[colname2idx['X_COORD_CD']])
X_COORD_CD.map(checkNull).countByValue()

defaultdict(int, {'NULL': 188146, 'VALID': 4913085})

## Date Variables

### CMPLNT_FR_DT, CMPLNT_FR_TM, CMPLNT_TO_DT and CMPLNT_TO_TM
- Check Nullity of from
- Check date format using checkDate() in utilities.py
- Comebine date and time to form two datetime object(from and to) for each row.
- Check whether 'from' is earilier than 'to'

In [28]:
%run ./crime-data-process/code/utilities.py

In [29]:
datetime_format = date_format+' '+time_format
datetime_format

'%m/%d/%Y %H:%M:%S'

In [171]:
alist = [1,2,3,4]
alist[:2] = [0,0]
alist

[0, 0, 3, 4]

In [176]:
# Implementation
# check_CMPLNT_TO check both CMPLNT_TO_DT and CMPLNT_TO_TM at the same time
# It return the validity of both field
def check_FR_TO(from_date,from_time,to_date,to_time,date_format,time_format):
    ###########
    #Null check
    ###########
    individual_checks = [checkDate(from_date),checkTime(from_time),checkDate(to_date),checkTime(to_time)]
    if 'NULL' in individual_checks:
        return individual_checks
#         if 'NULL' in individual_checks[:2]:
#             individual_checks = ['NULL','NULL','NULL','NULL']
#             return individual_checks
#         if 'NULL' in individual_checks[2:4]:
#             return individual_checks
    #Check for invalidity
    #If any field format is invalid return individual check
    elif 'INVALID' in individual_checks:
        return individual_checks
    else:
        #ALL VALID for individual check
        ###Start combining
        #Define datetime format
        datetime_format = date_format+' '+time_format
        from_datetime = datetime.datetime.strptime(from_date+' '+from_time,datetime_format)
        to_datetime = datetime.datetime.strptime(from_date+' '+from_time,datetime_format)
        if from_datetime>to_datetime:
            return ['VALID','VALID','INVALID','INVALID'] 
        else:
            return individual_checks

In [177]:
# Testing
print ','.join(check_FR_TO('10/09/1992','12:00:00','10/09/1991','12:00:00',date_format,time_format))
print check_FR_TO('','','10/09/1991','12:00:00',date_format,time_format)
print check_FR_TO('10/09/1991','12:00:00','','',date_format,time_format)

INVALID,VALID,INVALID,VALID
['NULL', 'NULL', 'INVALID', 'VALID']
['INVALID', 'VALID', 'NULL', 'NULL']


In [178]:
from_to_tuple = lines.map(lambda x:x[1:5])
checkResults_from_to = from_to_tuple.map(lambda x: check_FR_TO(x[0],x[1],x[2],x[3],date_format,time_format))\
.map(lambda x: ','.join(x))

In [179]:
checkResults_from_to.countByValue()

defaultdict(int,
            {'INVALID,INVALID,INVALID,INVALID': 2,
             'INVALID,INVALID,INVALID,VALID': 3,
             'INVALID,INVALID,NULL,NULL': 1,
             'INVALID,INVALID,VALID,INVALID': 5,
             'INVALID,INVALID,VALID,VALID': 8,
             'INVALID,VALID,INVALID,INVALID': 42,
             'INVALID,VALID,INVALID,NULL': 1,
             'INVALID,VALID,INVALID,VALID': 1696,
             'INVALID,VALID,NULL,NULL': 1920,
             'INVALID,VALID,NULL,VALID': 16,
             'INVALID,VALID,VALID,INVALID': 32,
             'INVALID,VALID,VALID,NULL': 2,
             'INVALID,VALID,VALID,VALID': 4287,
             'NULL,INVALID,VALID,VALID': 1,
             'NULL,NULL,NULL,NULL': 1,
             'NULL,VALID,INVALID,INVALID': 1,
             'NULL,VALID,INVALID,VALID': 1,
             'NULL,VALID,NULL,NULL': 137,
             'NULL,VALID,NULL,VALID': 9,
             'NULL,VALID,VALID,INVALID': 2,
             'NULL,VALID,VALID,VALID': 503,
             'VALID,I

### RPD_DT
- Check Nullity
- Check date format using checkDate()

In [40]:
def check_RPD_DT(line):
    if checkNull(line)=='VALID':
        return checkDate(line)
    else:
        return 'NULL'
RPD_DT = lines.map(lambda x:x[colname2idx['RPT_DT']])
checkResults_RPD_DT = RPD_DT.map(lambda x: check_RPD_DT(x))
checkResults_RPD_DT.countByValue()

defaultdict(int, {'VALID': 5101231})

## Categorical Variables

### KY_CD and PD_CD (3 digits codes)
- Check nullity
- Use checkRegex() to check whether it consists of 3-digits only.

In [48]:
def check_KY_CD(line,regex):
    if checkNull(line)=='VALID':
        return checkRegex(line,regex)
    else:
        return 'NULL'
regex_3_digits = '^\d{3}$'

In [49]:
KY_CD = lines.map(lambda x:x[colname2idx['KY_CD']])
PD_CD = lines.map(lambda x:x[colname2idx['PD_CD']])
checkResults_KY_CD = KY_CD.map(lambda x: check_KY_CD(x,regex_3_digits))
checkResults_PD_CD = PD_CD.map(lambda x: check_KY_CD(x,regex_3_digits))

In [50]:
checkResults_KY_CD.countByValue()

defaultdict(int, {'VALID': 5101231})

### OFNS_DESC and PD_DESC (Description of codes)
- Check nullity only

In [52]:
checkResults_OFNS_DESC = lines.map(lambda x: x[colname2idx['OFNS_DESC']]).map(checkNull)
checkResults_OFNS_DESC.countByValue()

defaultdict(int, {'NULL': 18840, 'VALID': 5082391})

In [234]:
checkResults_PD_DESC = lines.map(lambda x: x[colname2idx['PD_DESC']]).map(checkNull)
checkResults_PD_DESC.countByValue()

defaultdict(int, {'NULL': 4574, 'VALID': 5096657})

#### Data Quality Issue
OFNS_DESC should be a one to one map with KY_CD (similarly, PD_DESC and PD_CD) but the situations that one code maps to different description exsists. (Usually minor difference)

Storing replicated descriptions is not wise and consumes unnecessary storage. Instead we should make a table of code-description mapping just as what shown below.

In [90]:
KY_CD_DESC = lines.map(lambda x:x[6]+'/t'+x[7])
KY_CD_count = KY_CD_DESC.countByValue()

In [196]:
KY_CD_count

defaultdict(int,
            {u'101/tMURDER & NON-NEGL. MANSLAUGHTER': 4574,
             u'102/tHOMICIDE-NEGLIGENT-VEHICLE': 93,
             u'103/tHOMICIDE-NEGLIGENT,UNCLASSIFIE': 33,
             u'104/tRAPE': 13791,
             u'105/t': 2,
             u'105/tROBBERY': 198772,
             u'106/t': 55,
             u'106/tFELONY ASSAULT': 184069,
             u'107/t': 1,
             u'107/tBURGLARY': 191406,
             u'109/t': 9,
             u'109/tGRAND LARCENY': 429196,
             u'110/tGRAND LARCENY OF MOTOR VEHICLE': 102061,
             u'111/t': 1,
             u'111/tPOSSESSION OF STOLEN PROPERTY': 9112,
             u'112/t': 6,
             u'112/tTHEFT-FRAUD': 56762,
             u'113/t': 1,
             u'113/tFORGERY': 49303,
             u'114/tARSON': 13984,
             u'115/tPROSTITUTION & RELATED OFFENSES': 122,
             u'116/t': 2,
             u'116/tSEX CRIMES': 10853,
             u'117/t': 4,
             u'117/tDANGEROUS DRUGS': 62679,
  

In [123]:
# Create new table that maps KY_CD code to unique description
import collections
KY_CD_map = collections.defaultdict()
for key in KY_CD_count.keys():
    pairs = key.split('/t')
    if len(pairs[1])==0:
        continue
    else:
        code = pairs[0]
        description = pairs[1]
        value = KY_CD_count[key]
        if code in KY_CD_map.keys():
            #print('code %s replicate!'%(code))
            if value>KY_CD_map[code]:
                KY_CD_map[code]=description
        else:
            KY_CD_map[code]=description      

In [125]:
KY_CD_map_df = pd.DataFrame(KY_CD_map.values(),index=KY_CD_map.keys(),columns=['DESCRIPTION']).sort_index()

In [156]:
# The result dict is one-one
KY_CD_map_df.head()

,DESCRIPTION
101,MURDER & NON-NEGL. MANSLAUGHTER
102,HOMICIDE-NEGLIGENT-VEHICLE
103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
104,RAPE
105,ROBBERY


### CRM_ATPT_CPTD_CD, LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC
These columns are catogorical variables which only consists of a few cases. Our procedure will:
- Check nullity
- Check whether a line match one of the cases

The functions used to check validities are all stores in utilities.py

In [131]:
CRM_ATPT_CPTD_CD = lines.map(lambda x: x[colname2idx['CRM_ATPT_CPTD_CD']])
#Use the following method to check validity
def checkCaseStatus(line):
    # check the validation of column offense descrption
    if checkNull(line) == 'VALID':
        if (line == 'COMPLETED' or line == 'ATTEMPTED'):
            return 'VALID'
        else:
            return 'INVALID'
    else: 
        return 'NULL'

In [133]:
checkResults_CRM_ATPT_CPTD_CD = CRM_ATPT_CPTD_CD.map(checkCaseStatus)
checkResults_CRM_ATPT_CPTD_CD.countByValue()

defaultdict(int, {'NULL': 7, 'VALID': 5101224})

In [235]:
LAW_CAT_CD = lines.map(lambda x: x[colname2idx['LAW_CAT_CD']])
checkResults_LAW_CAT_CD = LAW_CAT_CD.map(checkLevelOfOffense)
checkResults_LAW_CAT_CD.countByValue()

defaultdict(int, {'VALID': 5101231})

In [240]:
JURIS_DESC = lines.map(lambda x:x[colname2idx['JURIS_DESC']])
JURIS_DESC.countByValue()

defaultdict(int,
            {u'AMTRACK': 153,
             u'CONRAIL': 14,
             u'DEPT OF CORRECTIONS': 4825,
             u'DISTRICT ATTORNEY OFFICE': 1,
             u'FIRE DEPT (FIRE MARSHAL)': 514,
             u'HEALTH & HOSP CORP': 2590,
             u'LONG ISLAND RAILRD': 439,
             u'METRO NORTH': 531,
             u'N.Y. HOUSING POLICE': 390853,
             u'N.Y. POLICE DEPT': 4538344,
             u'N.Y. STATE PARKS': 272,
             u'N.Y. STATE POLICE': 1209,
             u'N.Y. TRANSIT POLICE': 108817,
             u'NEW YORK CITY SHERIFF OFFICE': 134,
             u'NYC DEPT ENVIRONMENTAL PROTECTION': 14,
             u'NYC PARKS': 71,
             u'NYS DEPT ENVIRONMENTAL CONSERVATION': 1,
             u'NYS DEPT TAX AND FINANCE': 77,
             u'OTHER': 13575,
             u'POLICE DEPT NYC': 8986,
             u'PORT AUTHORITY': 24657,
             u'SEA GATE POLICE DEPT': 30,
             u'STATN IS RAPID TRANS': 306,
             u'TRI-BORO BRD

In [242]:
BORO_NM = lines.map(lambda x: x[colname2idx['BORO_NM']])
BORO_NM.countByValue()

defaultdict(int,
            {u'': 463,
             u'BRONX': 1103514,
             u'BROOKLYN': 1526213,
             u'MANHATTAN': 1216249,
             u'QUEENS': 1011002,
             u'STATEN ISLAND': 243790})

In [243]:
ADDR_PCT_CD = lines.map(lambda x: x[colname2iADDR_PCT_CDDR_PCT_CD_PCT_CD'ADDR_PCT_CD']])
ADDR_PCT_CD.countByValue()

defaultdict(int,
            {u'': 390,
             u'1': 61537,
             u'10': 43479,
             u'100': 25879,
             u'101': 41139,
             u'102': 65302,
             u'103': 80404,
             u'104': 75116,
             u'105': 74018,
             u'106': 61495,
             u'107': 53750,
             u'108': 54117,
             u'109': 80169,
             u'110': 68583,
             u'111': 33116,
             u'112': 36889,
             u'113': 87842,
             u'114': 91696,
             u'115': 81493,
             u'120': 109720,
             u'121': 17449,
             u'122': 82693,
             u'123': 34003,
             u'13': 74443,
             u'14': 119415,
             u'17': 32659,
             u'18': 79129,
             u'19': 65983,
             u'20': 40974,
             u'22': 4420,
             u'23': 66693,
             u'24': 50509,
             u'25': 67480,
             u'26': 34275,
             u'28': 53052,
             u'30': 49

In [244]:
LOC_OF_OCCUR_DESC = lines.map(lambda x: x[colname2idx['LOC_OF_OCCUR_DESC']])
LOC_OF_OCCUR_DESC.countByValue()

defaultdict(int,
            {u'': 1127128,
             u' ': 213,
             u'FRONT OF': 1189787,
             u'INSIDE': 2527543,
             u'OPPOSITE OF': 140606,
             u'OUTSIDE': 2765,
             u'REAR OF': 113189})

In [245]:
PREM_TYP_DESC = lines.map(lambda x: x[colname2idx['PREM_TYP_DESC']])
PREM_TYP_DESC.countByValue()

defaultdict(int,
            {u'': 33279,
             u'ABANDONED BUILDING': 1891,
             u'AIRPORT TERMINAL': 14315,
             u'ATM': 4267,
             u'BANK': 21252,
             u'BAR/NIGHT CLUB': 57065,
             u'BEAUTY & NAIL SALON': 13027,
             u'BOOK/CARD': 1647,
             u'BRIDGE': 5490,
             u'BUS (NYC TRANSIT)': 10581,
             u'BUS (OTHER)': 2638,
             u'BUS STOP': 4455,
             u'BUS TERMINAL': 3618,
             u'CANDY STORE': 6487,
             u'CEMETERY': 668,
             u'CHAIN STORE': 95079,
             u'CHECK CASHING BUSINESS': 5587,
             u'CHURCH': 9151,
             u'CLOTHING/BOUTIQUE': 42178,
             u'COMMERCIAL BUILDING': 131497,
             u'CONSTRUCTION SITE': 8411,
             u'DEPARTMENT STORE': 96971,
             u'DOCTOR/DENTIST OFFICE': 12253,
             u'DRUG STORE': 37479,
             u'DRY CLEANER/LAUNDRY': 10278,
             u'FACTORY/WAREHOUSE': 5522,
             u'

In [247]:
HADEVELOPT = lines.map(lambda x: x[colname2idx['HADEVELOPT']])
HADEVELOPT.countByValue()

defaultdict(int,
            {u'': 4848026,
             u'1010 EAST 178TH STREET': 1,
             u'1162-1176 WASHINGTON AVENUE': 318,
             u'131 SAINT NICHOLAS AVENUE': 232,
             u'1471 WATSON AVENUE': 224,
             u'303 VERNON AVENUE': 646,
             u'33-35 SARATOGA AVENUE': 365,
             u'344 EAST 28TH STREET': 1,
             u'45 ALLEN STREET': 111,
             u'572 WARREN STREET': 342,
             u'830 AMSTERDAM AVENUE': 432,
             u'ADAMS': 1930,
             u'ALBANY': 2004,
             u'ALBANY II': 1065,
             u'AMSTERDAM': 1095,
             u'ARMSTRONG I': 1395,
             u'ARMSTRONG II': 1538,
             u'ASTORIA': 1564,
             u'ATLANTIC TERMINAL SITE 4B': 657,
             u'AUDUBON': 362,
             u'BAILEY AVENUE-WEST 193RD STREET': 484,
             u'BAISLEY PARK': 337,
             u'BARUCH': 3147,
             u'BARUCH HOUSES ADDITION': 120,
             u'BAY VIEW': 1852,
             u'BAYCHESTER':

## String Columns
JURIS_DESC, PREM_TYP_DESC, PARKS_NM, HADEVELOPT
- Check Nullity Only